In [ ]:
%pip install --upgrade --quiet  lark
%pip install --upgrade --quiet  langchain-chroma

In [1]:

from qdrant_client.http.models import Distance, VectorParams
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.docstore.document import Document
from typing import List, Dict
from langchain.vectorstores import Redis
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain.chains.query_constructor.base import (
    get_query_constructor_prompt,
    load_query_constructor_runnable,
    AttributeInfo)
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.1,api_key= os.getenv('llm_api_1'))


/home/justtuananh/miniconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/justtuananh/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/justtuananh/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`

In [3]:


# Function to load and chunk data from a folder
def load_and_chunk_data(data_path):
    docs = []
    # Load all .txt files from the specified folder
    for filename in os.listdir(data_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(data_path, filename)
            loader = TextLoader(file_path, encoding='utf-8')
            docs.extend(loader.load())

    # Define headers to split on for Markdown splitting
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    # Initialize Markdown splitter
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=False
    )

    chunk_size = 512
    chunk_overlap = 0

    # Initialize character-based splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    chunked_docs = []

    # Process each document
    for doc in docs:
        # Split document by Markdown headers
        md_header_splits = markdown_splitter.split_text(doc.page_content)

        # Further split the Markdown header splits into smaller chunks
        chunked_docs.extend(text_splitter.split_documents(md_header_splits))

    return chunked_docs


data_path = '/home/justtuananh/AI4TUAN/DOAN2024/eval_rag_vietnamese/thongtintuyensinh'
chunked_data = load_and_chunk_data(data_path)


In [4]:
vectorstore = Chroma.from_documents(chunked_data, HF_EMBEDDING, persist_directory="./data_chroma")

# vectorstore = Chroma(persist_directory="./data_chroma" ,embedding_function=HF_EMBEDDING)
# retriever =vectorstore.as_retriever()


In [5]:
def build_index_schema(documents: List[Document]) -> Dict:
    schema = {"text": []}
    for doc in documents:
        for key in doc.metadata:
            name_dict = {"name": f"{key}"}
            if name_dict not in schema["text"]:
                schema["text"].append(name_dict)
    return schema

index_schema = build_index_schema(chunked_data)
print(index_schema)

{'text': [{'name': 'Header 1'}, {'name': 'Header 2'}, {'name': 'Header 3'}]}


In [6]:
retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 20}),
    return_source_documents=True
)


In [7]:
# add self-query
from langchain.chains.query_constructor.base import (
    get_query_constructor_prompt,
    load_query_constructor_runnable,
    AttributeInfo)

document_content_description = "The document contains introductory information about the admissions details of the Military Technical Academy."

headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]


def generate_metadata_desc(field_name: str, header_info: List[tuple], qa_retriever: RetrievalQA) -> str:
    query = f"""
    Given a list of tuples indicating all possible metadata fields for this article,
    provide a very brief description (15 words or less) of the specified field,
    including its position in the hierarchy of headers (Header 1, Header 2, Header 3, etc.).

    all fields {header_info}
    specified field: {field_name}
    """

    return qa_retriever({"query": query})["result"]

def build_metadata_field_info(schema: Dict, header_info: List[tuple], qa_retriever: RetrievalQA) -> List[AttributeInfo]:
    filter_instructions = \
        "ALWAYS filter with one or more CONTAINS comparators, and use the OR operator to check ALL other fields."\
        "if the value of this field contains a word or phrase that is very similar to a word or phrase in the query, " \
        "filter for the exact string from the value rather than the query."
    filter_h1 = \
        "if the query contains a misspelling or an alternate spelling of this name, "\
        "filter for the value of this field rather than the name in the query."\
        "the h1-level filter should ALWAYS be combined with subsection filters using an AND operator. \n"
    filter_exclusion = \
        " NEVER filter this field on the value of 'Header 1'."
    attr_info_list = []
    for field in schema["text"]:
        desc = generate_metadata_desc(field["name"], header_info, qa_retriever) + filter_instructions
        if field["name"] == "Header 1":
            new_attr = AttributeInfo(
                name=field["name"],
                description = filter_h1 + desc,
                type="string"
            )
        else:
            new_attr = AttributeInfo(
                name=field["name"],
                description = desc + filter_exclusion,
                type="string"
            )
        attr_info_list.append(new_attr)
    return attr_info_list

metadata_field_info = build_metadata_field_info(index_schema, headers_to_split_on, retriever)

/tmp/ipykernel_88527/2234936660.py:26: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  return qa_retriever({"query": query})["result"]


In [8]:
metadata_field_info

[AttributeInfo(name='Header 1', description='if the query contains a misspelling or an alternate spelling of this name, filter for the value of this field rather than the name in the query.the h1-level filter should ALWAYS be combined with subsection filters using an AND operator. \n"Introduction to the Military Technical Academy, top-level header."ALWAYS filter with one or more CONTAINS comparators, and use the OR operator to check ALL other fields.if the value of this field contains a word or phrase that is very similar to a word or phrase in the query, filter for the exact string from the value rather than the query.', type='string'),
 AttributeInfo(name='Header 2', description='"Method of recruitment, 2024 recruitment process details (Header 2)"ALWAYS filter with one or more CONTAINS comparators, and use the OR operator to check ALL other fields.if the value of this field contains a word or phrase that is very similar to a word or phrase in the query, filter for the exact string fr

In [9]:
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)
retriever.invoke("điểm chuẩn thi vào học viện năm 2024")

OutputParserException: Parsing text
```json
{
    "query": "điểm chuẩn thi vào học viện năm 2024",
    "filter": "or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(or(o

In [21]:
examples = [
    (
        "What is the admission quota for male candidates from the Northern region for the Military Technical Academy in 2024?",
        {"query": "admission quota, Military Technical Academy, 2024, male candidates from the Northern region",
         "filter": 'and(contain("Header 1", "ADMISSION INFORMATION FOR THE MILITARY TECHNICAL ACADEMY IN 2024"), or(contain("Header 2", "Admission quota"), '
                   'contain("Header 3", "Admission quota")))'}
    ),
    (
        "What are the admission methods at the Military Technical Academy?",
        {"query": "admission methods, Military Technical Academy",
         "filter": 'and(contain("Header 1", "ADMISSION INFORMATION FOR THE MILITARY TECHNICAL ACADEMY IN 2024"), or(contain("Header 2", "Admission methods"), '
                   'contain("Header 3", "Method 1: Direct admission"), contain("Header 3", "Method 2: Priority admission")))'}
    ),
    (
        "What was the admission score for the academy in 2023?",
        {"query": "admission score, 2023",
         "filter": 'and(contain("Header 1", "Admission information for 2021, 2022, 2023 of the Military Technical Academy"), or(contain("Header 2", "Admission scores for 2021, 2022, 2023"), '
                   'contain("Header 2", "Scores for 2023 of the academy"), contain("Header 2", "Cut-off scores for 2023")))'}
    )
]


In [23]:
prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
    examples=examples
)
result = prompt.format(query="{query}") 
result



'Your goal is to structure the user\'s query to match the request schema provided below.\n\n<< Structured Request Schema >>\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n```json\n{\n    "query": string \\ text string to compare to document contents\n    "filter": string \\ logical condition statement for filtering documents\n}\n```\n\nThe query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.\n\nA logical condition statement is composed of one or more comparison and logical operation statements.\n\nA comparison statement takes the form: `comp(attr, val)`:\n- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator\n- `attr` (string):  name of attribute to apply the comparison to\n- `val` (string): is the comparison value\n\nA logical operation statement takes the form `op(statement1, statement2, ...)

In [24]:
chain = load_query_constructor_runnable(
    llm=llm,
    attribute_info=metadata_field_info,
    document_contents=document_content_description,
    examples=examples,
    fix_invalid=True
)

In [27]:
query = "điểm chuẩn học viện năm 2021"
chain.invoke(({"query": query}))

StructuredQuery(query='điểm chuẩn học viện năm 2021', filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.CONTAIN: 'contain'>, attribute='Header 1', value='Admission information for 2021 of the Military Technical Academy'), Operation(operator=<Operator.OR: 'or'>, arguments=[Comparison(comparator=<Comparator.CONTAIN: 'contain'>, attribute='Header 2', value='Admission scores for 2021'), Comparison(comparator=<Comparator.CONTAIN: 'contain'>, attribute='Header 3', value='Cut-off scores for 2021')])]), limit=None)

In [28]:
sq_retriever = SelfQueryRetriever(
    query_constructor=chain,
    vectorstore=vectorstore,
    verbose=True,
)

In [29]:
sq_retriever

SelfQueryRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7756db8ea2d0>, query_constructor=FewShotPromptTemplate(input_variables=['query'], examples=[{'i': 1, 'user_query': 'What is the admission quota for male candidates from the Northern region for the Military Technical Academy in 2024?', 'structured_request': '{{\n    "query": "admission quota, Military Technical Academy, 2024, male candidates from the Northern region",\n    "filter": "and(contain(\\"Header 1\\", \\"ADMISSION INFORMATION FOR THE MILITARY TECHNICAL ACADEMY IN 2024\\"), or(contain(\\"Header 2\\", \\"Admission quota\\"), contain(\\"Header 3\\", \\"Admission quota\\")))"\n}}'}, {'i': 2, 'user_query': 'What are the admission methods at the Military Technical Academy?', 'structured_request': '{{\n    "query": "admission methods, Military Technical Academy",\n    "filter": "and(contain(\\"Header 1\\", \\"ADMISSION INFORMATION FOR THE MILITARY TECHNICAL ACADEMY IN 2024\\"), or(contain(\\"He

In [30]:
sq_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=sq_retriever,
    return_source_documents=True
)

In [31]:
query = "chỉ tiêu tuyển sinh của học viện năm 2023"
response = sq_qa({"query": query})
print(response)

ValueError: Received disallowed comparator Comparator.CONTAIN. Allowed comparators are [<Comparator.EQ: 'eq'>, <Comparator.NE: 'ne'>, <Comparator.GT: 'gt'>, <Comparator.GTE: 'gte'>, <Comparator.LT: 'lt'>, <Comparator.LTE: 'lte'>]